In [5]:
import json
import foursquareWithHosts
from datetime import datetime
from sys import stdout
import requests
with open('keys/appCredentialsSudo.json') as credentialsFile:    
    credentials = json.load(credentialsFile)
with open('keys/accessTokensSudo.json') as tokensFile:    
    accessTokens = json.load(tokensFile)
requests.packages.urllib3.disable_warnings()

In [6]:
clients = []
users = []
for accessToken in accessTokens[:5]:
    client = foursquareWithHosts.Foursquare(client_id=credentials['client_id'], client_secret=credentials['client_secret'], redirect_uri=credentials['redirect_uri'])
    client.set_access_token(accessToken)
    user = client.users()
    clients.append(client)
    users.append(user)

Hi
Hi
Hi
Hi
Hi


# Check and add new friends

In [7]:
def progress(i, n):
    stdout.write("\r%f%%" % (i*100/float(n)))
    stdout.flush()
    if i == n-1:
        stdout.write("\r100%")
        print("\r\n")

In [8]:
def getMongoDBInstance():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.habidatum
    return db

In [9]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
db = getMongoDBInstance()

In [10]:
def fillSourceDataFrames(folderName):
    checkinsDF = pd.read_csv('data/'+folderName+'checkins.tsv', sep='\t',encoding='utf-8')
    checkpointsDF = pd.read_csv('data/'+folderName+'checkPoints.tsv', sep='\t',encoding='utf-8')
    usersDF = pd.read_csv('data/'+folderName+'users.tsv', sep='\t',encoding='utf-8')
    return (checkinsDF, checkpointsDF, usersDF)

In [11]:
#Getting old  pending users
def getPendingFriends(db):
    existingPendingFriends = []
    existingPendingFriendsCoursor = db.pendingUsers.find()
    for pendingFriend in existingPendingFriendsCoursor:
        existingPendingFriends.append(int(pendingFriend['requestedUser']['user']['id']))
    return existingPendingFriends

In [12]:
def getUsersToFriend(usersDF, existingPendingFriends):
    return usersDF[~usersDF.userId.isin(existingPendingFriends)].userId.tolist()

In [13]:
def countUsersToFriend(usersToFriend):
    usersToFriendCount = len(usersToFriend)
    if usersToFriendCount:
        print('{} users to friend').format(usersToFriendCount)
    else:
        print('No new users to friend')
    return usersToFriendCount

In [14]:
def friendUsers(usersToFriend, db, clients):
    pendingUsers = []
    for index, userId in enumerate(usersToFriend):
        progress(index,len(usersToFriend))
        try:
            pendingUser = clients[index%len(clients)].users.request(userId)
            pendingUsers.append(pendingUser)
            db.pendingUsers.insert({'requestingUser':index%len(clients),
                                        'requestedUser':pendingUser,
                                         'requestedAt':datetime.now()})
        except:
            print 'User',index%len(clients), 'Can\'t friend'
    return pendingUsers

In [15]:
def addNewUsersFromCheckins(folderName, clients, db):
    (checkinsDF, checkpointsDF, usersDF) = fillSourceDataFrames(folderName)
    existingPendingFriends = getPendingFriends(db)
    usersToFriend = getUsersToFriend(usersDF, existingPendingFriends)
    if countUsersToFriend(usersToFriend):
        friendUsers(usersToFriend, db, clients)

# Run the loop

In [ ]:
import schedule
import time

schedule.clear()
schedule.every(2).minutes.do(addNewUsersFromCheckins,'LastJuneWeekend/', clients, db)

while True:
    schedule.run_pending()
    time.sleep(1)

In [28]:
addNewUsersFromCheckins('LastJuneWeekend/', clients, db)

55 users to friend
100%



In [22]:
clients[0].users.pending()

AttributeError: 'Users' object has no attribute 'pending'

In [26]:
existingPendingFriendsCoursor0 = db.pendingUsers.find({'requestingUser':4})

In [28]:
9%len(clients)

4

In [27]:
for user in enumerate(existingPendingFriendsCoursor0[:200]):
    try:
        clients[4].users.unfriend(user[1]['requestedUser']['user']['id'])
    except:
        'Already'

In [2]:
range(0,10)[-7:]

[3, 4, 5, 6, 7, 8, 9]